In [19]:
import pandas as pd
import numpy as np

In [20]:
csv_file = 'coordinates.csv'

# Read the CSV file into a Pandas DataFrame
df1 = pd.read_csv(csv_file)

In [21]:
df1

,latitude,longitude,altitude
0,-12.46341,130.84628,24.080944
1,-12.46330,130.84621,24.550274
2,-12.46320,130.84617,24.809553
3,-12.46299,130.84607,25.300398
4,-12.46291,130.84604,25.314518
...,...,...,...
305923,-34.92415,138.59327,39.912281
305924,-34.92394,138.59336,39.126549
305925,-34.92385,138.59338,38.784435
305926,-34.92373,138.59338,38.363876


In [22]:
# Convert latitude and longitude to float and calculate distance
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r * 1000

In [23]:
# Convert latitude and longitude columns to NumPy arrays
latitudes = df1['latitude'].values
longitudes = df1['longitude'].values

In [24]:
# Calculate the distance between consecutive points using vectorized operations
distances = haversine(latitudes[:-1], longitudes[:-1], latitudes[1:], longitudes[1:])

In [25]:
# Add the distances to the DataFrame
df1['distance'] = [np.nan] + list(distances)

# Calculate the slope between consecutive points
df1['slope'] = (df1['altitude'] - df1['altitude'].shift(-1)) / (df1['distance'])

In [26]:
# Create a new DataFrame with 'distance' and 'slope' columns
df2 = df1[['distance', 'slope']].copy()

In [27]:
df2

,distance,slope
0,NaN,NaN
1,14.400398,-0.018005
2,11.937530,-0.041118
3,25.751712,-0.000548
4,9.473184,-0.002485
...,...,...
305923,10.948065,0.071769
305924,24.750619,0.013822
305925,10.172302,0.041344
305926,13.343391,0.036922


In [28]:
# Remove the first and last rows
df2 = df2.iloc[1:-1]

# Reset the index of the new DataFrame
df2.reset_index(drop=True, inplace=True)

In [29]:
df2

,distance,slope
0,14.400398,-0.018005
1,11.937530,-0.041118
2,25.751712,-0.000548
3,9.473184,-0.002485
4,10.909284,0.007027
...,...,...
305921,11.404993,0.029913
305922,10.948065,0.071769
305923,24.750619,0.013822
305924,10.172302,0.041344


In [30]:
df2.loc[:, "slope_deg"] = np.arctan(df2['slope']*85.94)
df2

/tmp/ipykernel_3145/2162963385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[:, "slope_deg"] = np.arctan(df2['slope']*85.94)


,distance,slope,slope_deg
0,14.400398,-0.018005,-0.997050
1,11.937530,-0.041118,-1.295015
2,25.751712,-0.000548,-0.047088
3,9.473184,-0.002485,-0.210381
4,10.909284,0.007027,0.543275
...,...,...,...
305921,11.404993,0.029913,1.199811
305922,10.948065,0.071769,1.410063
305923,24.750619,0.013822,0.871069
305924,10.172302,0.041344,1.296446


In [31]:
df2 = df2.drop(columns=['slope'])

# Save the modified DataFrame to a CSV file
df2.to_csv('route.csv', index=False)